In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.pipeline import Pipeline ,make_pipeline
from sklearn import impute
from sklearn import compose
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, plot_confusion_matrix,roc_auc_score
from sklearn import set_config
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler,Normalizer,PowerTransformer,QuantileTransformer, RobustScaler,StandardScaler,LabelEncoder, OneHotEncoder,OrdinalEncoder
from sklearn.tree          import DecisionTreeClassifier
from sklearn.linear_model  import LogisticRegression    
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn
set_config(display='diagram') # Useful for display the pipeline
print("Pandas  ", pd.__version__)
print("Sklearn ", sklearn.__version__) # Try to use 0.24



Pandas   1.2.4
Sklearn  0.24.2


In [2]:
data = pd.read_csv('./data/data_5secondWindow .csv')
data.head(5)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,Luca
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,andrea


In [ ]:
data.describe()

In [3]:
from random import shuffle

In [21]:
l = data.user.unique().tolist()
l

['Luca',
 'andrea',
 'Federica',
 'michelangelo',
 'Damiano',
 'Claudio',
 'Vincenzo',
 'Serena',
 'Pierpaolo',
 'IvanHeibi',
 'AndreaCarpineti',
 'Elena',
 'Riccardo']

In [54]:
shuffle(l)


# Create test data 

In [74]:
# create test data from some user 
def split_data(df,l):
   
    # create random list of indi
    train_user=l[1:]
    test_user=l[:1]
    # get splitting indicies
    train=data[data['user'].isin(train_user)]
    test = data[data['user'].isin(test_user)]
    return train, test

In [76]:
shuffle(l)
df_train,df_test=split_data(data,l)
unimp    = ['id','user','time']
df_train = df_train.drop(unimp,axis=1)
df_test  = df_test.drop(unimp,axis=1)
print(df_train.shape)
print(df_test.shape)

(5622, 67)
(271, 67)


In [84]:
df_test

,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,...,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target
49,NaN,100.0,10.413624,3.324387,14.608536,4.031835,0.995565,0.995565,0.995565,0.004182,...,NaN,77.221125,77.221125,77.221125,NaN,NaN,NaN,NaN,NaN,Walking
56,NaN,3.0,9.918021,5.371977,15.088214,3.438894,0.999042,0.997809,0.999737,0.001071,...,NaN,79.330518,78.588379,80.072657,1.049543,1.200,1.20,1.20,NaN,Walking
61,NaN,92.0,10.397982,2.866489,15.540415,4.675462,0.980540,0.962893,0.989908,0.012693,...,6.363961,78.195398,78.195398,78.195398,3.664411,0.395,0.39,0.40,0.007071,Walking
90,NaN,100.0,9.117016,4.277544,13.888088,3.758204,0.823188,0.823188,0.823188,0.000227,...,NaN,82.598307,82.598307,82.598307,NaN,1.740,1.74,1.74,NaN,Walking
94,NaN,100.0,10.683149,2.207763,14.429910,4.189774,0.962968,0.940205,0.985731,0.032191,...,NaN,76.875864,76.875864,76.875864,NaN,1.710,1.71,1.71,NaN,Walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5759,NaN,3.0,10.190591,5.715392,14.813796,3.931166,0.971048,0.940473,0.994049,0.027579,...,NaN,86.247356,86.247356,86.247356,NaN,1.200,1.20,1.20,NaN,Walking
5779,NaN,100.0,10.074729,3.764139,13.486513,3.110941,0.998394,0.996488,0.999452,0.001307,...,NaN,81.107567,81.107567,81.107567,0.322535,1.130,1.13,1.13,NaN,Walking
5852,NaN,92.0,11.187854,6.020127,14.333121,3.055832,0.950652,0.942595,0.962863,0.007540,...,NaN,80.372346,80.372346,80.372346,1.688938,1.660,1.66,1.66,NaN,Walking
5875,NaN,15.0,9.497735,9.455685,9.543818,0.040397,0.645792,0.645769,0.645816,0.000033,...,NaN,56.028074,56.028074,56.028074,NaN,NaN,NaN,NaN,NaN,Still


In [77]:
print(df_train.shape)
print(df_test.shape)

(5622, 67)
(271, 67)


In [10]:
def fill_nan_with_mean_training(training, test):
    trainingFill = training.copy()
    testFill = test.copy()
    trainingFill = trainingFill.fillna(trainingFill.mean())
    trainingFill = trainingFill.fillna(0)
    testFill = testFill.fillna(trainingFill.mean())
    testFill = testFill.fillna(0)
    return trainingFill, testFill

def scale_features(train, test):
    # build scaler to apply on training and test the same transformation
    scaler = StandardScaler().fit(train)
    train_features_scaled = scaler.transform(train)
    test_features_scaled = scaler.transform(test)
    return train_features_scaled, test_features_scaled

In [78]:
df_train_fill,df_test_fill=fill_nan_with_mean_training(df_train,df_test)

In [79]:
X=df_train_fill.drop('target',axis=1)
y=df_train_fill.target
X_test=df_test_fill.drop('target',axis=1)

y_test=df_test_fill.target
X_sc,X_test_sc=scale_features(X, X_test)

# Pipeline 

In [ ]:
# num_attribs=df_train.drop('target',axis=1).columns.to_list()
# num_pip=Pipeline([('imputer',impute.SimpleImputer(strategy='median')),
#                   ('scalar',MinMaxScaler()),
#                ])  # ('PCA',PCA(n_components=12))
# preprocessor=compose.ColumnTransformer([
#     ('num',num_pip,num_attribs)
# ])

In [16]:
# pipeline for model
classifier_models = {
"DecisionTreeClassifier": DecisionTreeClassifier(),
'KNeighborsClassifier':KNeighborsClassifier(6),
'LogisticRegression':LogisticRegression(),
"RandomForestClassifier":RandomForestClassifier(),
"GradientBoostingClassifier":GradientBoostingClassifier(),
"XGBClassifier":XGBClassifier(),
"LGBMClassifier":LGBMClassifier(),
'GaussianNB':GaussianNB(),

}
# make pipline with  preprocessing 
classifier_models = {name: make_pipeline( model) for name, model in classifier_models.items()}
classifier_models["GradientBoostingClassifier"]

Pipeline(steps=[('gradientboostingclassifier', GradientBoostingClassifier())])

In [38]:
x_train, x_val, y_train, y_val = train_test_split(X_sc, y,test_size=0.4,stratify = y,random_state=10 )
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    valid_pred = model.predict(x_val)
    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_val, valid_pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_val, valid_pred)*100,
                    
                              "Time":     total_time},
                              ignore_index=True)
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


[15:38:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,99.047141,99.025332,3.487152
2,XGBClassifier,98.696088,98.679681,4.990760
3,RandomForestClassifier,98.445336,98.413381,1.302627
4,GradientBoostingClassifier,98.395186,98.352888,20.763938
5,KNeighborsClassifier,94.282849,94.255064,0.004057
6,DecisionTreeClassifier,94.132397,94.095578,0.150912
7,LogisticRegression,85.807422,85.748511,0.557232
8,GaussianNB,68.204614,66.958617,0.006358


# Best model for test data  

In [80]:
best_model=LGBMClassifier()
name='LGBMClassifier'

In [81]:
clf=Pipeline([('classification',best_model)])
test_result = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
start_time = time.time()
clf.fit(X_sc, y)
total_time = time.time() - start_time
pred = clf.predict(X_test_sc)
test_result = test_result.append({"Model":    name,
                          "Accuracy": accuracy_score(y_test, pred)*100,
                          "Bal Acc.": balanced_accuracy_score(y_test, pred)*100,
                          "Time":     total_time},
                          ignore_index=True)
test_result_ord = test_result.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
test_result_ord.index += 1 
test_result_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')



,Model,Accuracy,Bal Acc.,Time
1,LGBMClassifier,89.298893,66.777778,4.352133


In [82]:
print(classification_report(y_test, pred))
print()
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

         Bus       0.00      0.00      0.00         0
         Car       0.92      0.75      0.83        16
       Still       0.89      0.27      0.41        30
       Train       0.00      0.00      0.00         0
     Walking       0.99      0.99      0.99       225

    accuracy                           0.89       271
   macro avg       0.56      0.40      0.45       271
weighted avg       0.98      0.89      0.92       271


[[  0   0   0   0   0]
 [  1  12   0   2   1]
 [  3   0   8  18   1]
 [  0   0   0   0   0]
 [  1   1   1   0 222]]


# with optuna paramater

In [ ]:

m_=LGBMClassifier()
model_m=Pipeline([('preprocess',preprocessor),('classification',m_)])

In [ ]:
model_m.fit(X,y)

In [ ]:
pred_=model_.predict(X_test)
print(f"Accuracy {accuracy_score(y_test, pred_)*100}")
print(classification_report(y_test, pred_))
print()
print(confusion_matrix(y_test, pred_))

#  Drop some feature from target  beacuse of confusion materix result 

## Drop car  category 

In [ ]:
df_test_1   =df_test[df_test['target']!='Car']
df_train_1  =df_train[df_train['target']!='Car']

In [ ]:
print(df_test.shape)
print(df_train.shape)

# How much info reduced 

In [ ]:
print(f" After drop Car category we  reduce \n test data {100-df_test_1.shape[0]/df_test.shape[0]*100} % \n train data {100-df_train_1.shape[0]/df_train.shape[0]*100} %")

In [ ]:
print(df_train_1.shape)
print(df_test_1.shape)

In [ ]:
X_=df_train_1.drop('target',axis=1)
y_=df_train_1.target
X_test_=df_test_1.drop('target',axis=1)
y_test_=df_test_1.target

# train data 

In [ ]:
x_train_, x_val_, y_train_, y_val_ = train_test_split(X_, y_,test_size=0.4,stratify = y_,random_state=10 )
result_new = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [],'Time': []})
for model_name, model in classifier_models.items():
    start_time_ = time.time()
    model.fit(x_train_, y_train_)
    total_time_= time.time() - start_time_
    valid_pred_new = model.predict(x_val_)
    result_new = result_new.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_val_, valid_pred_new)*100,
                              "Bal Acc.": balanced_accuracy_score(y_val_, valid_pred_new)*100,
                              "Time":     total_time_},
                              ignore_index=True)
result_new_ord = result_new.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
result_new_ord.index += 1 
result_new_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

#  The highest accuracy model selected  manual tuning 

In [ ]:

best_=LGBMClassifier(boosting_type='gbdt', num_leaves=60, max_depth=6, learning_rate=0.07,
                          n_estimators=300, subsample_for_bin=200000, objective=None, class_weight=None,
                          min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                          subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, 
                          random_state=None, n_jobs=- 1, silent=True, importance_type='split')
model_=Pipeline([('preprocess',preprocessor),('classification',best_)])

In [ ]:
model_.fit(X_,y_)
pred_test_1=model_.predict(X_test_)

In [ ]:
accuracy_score(y_test_, pred_test_1)*100,

In [ ]:
print(classification_report(y_test_, pred_test_1))
print()
print(confusion_matrix(y_test_, pred_test_1))

# Optuna find best parameter for selected model

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
import optuna

In [ ]:
oth=Pipeline([('preprocess',preprocessor),('classification',GaussianNB())])

In [ ]:
 n_estimators = trial.suggest_int('n_estimators', 2, 20)
max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))

In [ ]:
import optuna

def objective(trial):

    boosting_type = trial.suggest_categorical("boosting_type", ["gbdt", "rf"])
    max_depth = trial.suggest_int('max_depth', 1, 32)
    n_estimators = trial.suggest_int("n_estimators", 100,500)
    learning_rate=trial.suggest_float("learning_rate", 0.001,0.1)
    min_data_in_leaf= trial.suggest_int("min_data_in_leaf", 10,500)
    
    lg_lgbm = LGBMClassifier( boosting_type=boosting_type, max_depth=max_depth, 
            n_estimators=n_estimators,learning_rate=learning_rate,min_data_in_leaf=min_data_in_leaf
        )

    score = cross_val_score(lg_lgbm, X_,y_, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))


# With optuna best parameter

 the best accuracy  **91.18993135011442**  in max_depth= 5, n_estimators= 300, num_leaves=70,min_data_in_leaf=60,learning_rate= 0.069

In [ ]:

lgb_af_=LGBMClassifier( max_depth= 5
, n_estimators= 300, num_leaves=70,min_data_in_leaf=60,learning_rate= 0.069)
model_end=Pipeline([('preprocess',preprocessor),('classification',lgb_af_)])

In [ ]:
model_end.fit(X_,y_)
pred_test_end=model_end.predict(X_test_)

In [ ]:
print(f"Accuracy {accuracy_score(y_test_, pred_test_end)*100}")
print(classification_report(y_test_, pred_test_end))
print()
print(confusion_matrix(y_test_, pred_test_end))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3e7089f3-11a5-48ad-89c4-39a166311a14' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>